In [1]:
!pip install pandas openpyxl dhlab

     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ------------------------- ------------ 41.0/60.9 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 60.9/60.9 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/204.6 kB ? eta -:--:--
     ------------------------------ ------- 163.8/204.6 kB 5.0 MB/s eta 0:00:01
     -------------------------------------- 204.6/204.6 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/60.


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from dhlab import nbtext
import re

# Les Excel-arket
naob_file = "NAOB 2001-05.xlsx"
df = pd.read_excel(naob_file, names=["Publikasjon", "Årstall"])


# Anta at kolonnen med publikasjonsinformasjon heter 'Publikasjon'
def parse_publication(pub_string):
    match = re.match(r'(.*?) – (.*?) \((.*?) (\d{4})\)', pub_string)
    if match:
        return {
            'Forfatter': match.group(1),
            'Tittel': match.group(2),
            'Sted': match.group(3),
            'År': match.group(4)
        }
    return {}

# Anvend parsing-funksjonen på hver rad
df = pd.concat([df, df['Publikasjon'].apply(parse_publication).apply(pd.Series)], axis=1)

# Funksjon for å hente URN
def get_urn(row):
    query = f"{row['Tittel']} {row['Forfatter']} {row['År']}"
    results = nbtext.book_search(query)
    if results and len(results) > 0:
        return results[0].get('urn', '')
    return ''

# Hent URN for hver publikasjon
df['URN'] = df.apply(get_urn, axis=1)

# Lag fullstendig URL
df['URL'] = 'https://urn.nb.no/' + df['URN']

# Lagre resultatet til en ny Excel-fil
df.to_excel('publikasjoner_med_url.xlsx', index=False)

AttributeError: module 'dhlab.legacy.nbtext' has no attribute 'book_search'

In [21]:
import pandas as pd
import  dhlab as dh
import re

# Les Excel-arket
df = pd.read_excel(naob_file, names=["Publikasjon", "Årstall"])

import re

def parse_publication(pub_string):
    match = re.match(r'(.*?) – (.*?) \((.*?) (\d{4})\)', pub_string)
    if match:
        forfatter = match.group(1)
        full_tittel = match.group(2)
        sted = match.group(3)
        år = match.group(4)
        
        # Del opp tittel og undertittel ved siste forekomst av ': '
        tittel_deler = full_tittel.rsplit(': ', 1)
        tittel = tittel_deler[0].strip()
        undertittel = tittel_deler[1].strip() if len(tittel_deler) > 1 else ''
        
        return {
            'Forfatter': forfatter,
            'Tittel': tittel,
            'Undertittel': undertittel,
            'Sted': sted,
            'År': år
        }
    return {}

# Anvend parsing-funksjonen på hver rad
df = pd.concat([df, df['Publikasjon'].apply(parse_publication).apply(pd.Series)], axis=1)



In [24]:
## Test

examples = [
    "Forfatter – Hovedtittel: Undertittel (Oslo 2001)",
    "Forfatter – Kompleks tittel: med flere: kolon: Faktisk undertittel (Oslo 2001)",
    "Forfatter – Tittel uten undertittel (Oslo 2001)"
]

for example in examples:
    result = parse_publication(example)
    print(f"Original: {example}")
    print(f"Parsed: {result}\n")


Original: Forfatter – Hovedtittel: Undertittel (Oslo 2001)
Parsed: {'Forfatter': 'Forfatter', 'Tittel': 'Hovedtittel', 'Undertittel': 'Undertittel', 'Sted': 'Oslo', 'År': '2001'}

Original: Forfatter – Kompleks tittel: med flere: kolon: Faktisk undertittel (Oslo 2001)
Parsed: {'Forfatter': 'Forfatter', 'Tittel': 'Kompleks tittel: med flere: kolon', 'Undertittel': 'Faktisk undertittel', 'Sted': 'Oslo', 'År': '2001'}

Original: Forfatter – Tittel uten undertittel (Oslo 2001)
Parsed: {'Forfatter': 'Forfatter', 'Tittel': 'Tittel uten undertittel', 'Undertittel': '', 'Sted': 'Oslo', 'År': '2001'}



In [25]:
import pandas as pd
import numpy as np

# Etter at du har anvendt parse_publication funksjonen

# Funksjon for å lage søkestreng
def create_search_string(row):
    parts = []
    if pd.notna(row['Tittel']) and row['Tittel']:
        parts.append(row['Tittel'])
    if pd.notna(row['Undertittel']) and row['Undertittel']:
        parts.append(row['Undertittel'])
    if pd.notna(row['Forfatter']) and row['Forfatter']:
        parts.append(row['Forfatter'])
    if pd.notna(row['År']) and row['År']:
        parts.append(str(row['År']))
    return ' '.join(parts)

# Lag søkestrenger for hver publikasjon
df['Søkestreng'] = df.apply(create_search_string, axis=1)

# Fjern eventuelle rader der søkestrengen er tom
df = df[df['Søkestreng'].notna() & (df['Søkestreng'] != '')]


# Opprett et Corpus-objekt med alle søkestrenger
corpus = dh.Corpus(df['Søkestreng'].tolist())


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:

# Hent metadata for corpus
metadata = corpus.metadata

# Funksjon for å finne beste treff
def find_best_match(row, metadata):
    search_string = row['Søkestreng'].lower()
    best_match = None
    best_score = 0
    
    for item in metadata:
        title = item.get('title', '').lower()
        authors = ' '.join(item.get('authors', [])).lower()
        year = str(item.get('year', ''))
        
        item_string = f"{title} {authors} {year}"
        
        # Enkel poengberegning basert på hvor mange ord som matcher
        score = sum(1 for word in search_string.split() if word in item_string)
        
        if score > best_score:
            best_score = score
            best_match = item
    
    return best_match['urn'] if best_match else ''

# Finn URN for hver publikasjon
df['URN'] = df.apply(lambda row: find_best_match(row, metadata), axis=1)

# Lag fullstendig URL
df['URL'] = 'https://urn.nb.no/' + df['URN']

# Lagre resultatet til en ny Excel-fil
df.to_excel('publikasjoner_med_url.xlsx', index=False)
